In [ ]:
# https://v6.exchangerate-api.com/v6/{API_KEY}/pair/USD/INR
import os
API_KEY = os.getenv("EXCHANGE_RATE_API_KEY")

In [2]:
# from langchain.agents.tool_calling import create_tool_calling_agent
from langchain.agents import create_agent 
from langchain_classic.agents import tool_calling_agent
from langchain.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI 
from langchain_core.messages import HumanMessage
import requests
from dotenv import load_dotenv


load_dotenv()

True

In [12]:
@tool
def get_conversion_rate(base_currency:str, target_currency: str) -> float:
	"""Get the conversion rate between two currencies."""
	
	url = f"https://v6.exchangerate-api.com/v6/{API_KEY}/pair/{base_currency}/{target_currency}"
	response = requests.get(url)
	data = response.json()
	return data["conversion_rate"]

In [13]:
@tool
def converter(base_currency: float, conversion_rate: float) -> float:
	""" from provided conversion rate this function calculates target currency value from a given base currency."""
	return base_currency * conversion_rate

In [14]:
google_model = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.3)

In [ ]:
agent = create_agent(
	google_model,
	tools=[get_conversion_rate, converter],
	system_prompt=(
        "You are a helpful assistant that converts currencies. Analyze the query and fill the base currency and target currency."
        "To answer a query, first use get_conversion_rate(base_currency, target_currency) "
        "to get the rate, then use converter(amount, conversion_rate) to get the result. "
        "Always return the final converted value."
    ),
)


In [16]:
query = "Convert 100 USD to INR"
messages = [HumanMessage(content=query)]

In [18]:
response = agent.invoke(
    input={"messages": messages}
    )

In [21]:
print(response['messages'][-1].content)

100 USD is 8864.96 INR.


In [ ]:
"""
{'messages': [HumanMessage(content='Convert 100 USD to INR', additional_kwargs={}, response_metadata={}, id='3b8ca94d-af5b-406a-88be-170483fd6470'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_conversion_rate', 'arguments': '{"base_currency": "USD", "target_currency": "INR"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--8049daaf-9ee3-49e0-95f5-3749dfde6735-0', tool_calls=[{'name': 'get_conversion_rate', 'args': {'base_currency': 'USD', 'target_currency': 'INR'}, 'id': '5130b1c9-fba5-41fe-850c-6e07692f28fc', 'type': 'tool_call'}], usage_metadata={'input_tokens': 125, 'output_tokens': 13, 'total_tokens': 138, 'input_token_details': {'cache_read': 0}}),
  ToolMessage(content='88.6496', name='get_conversion_rate', id='9f49c507-d6d5-4aa3-90a3-43875c421f62', tool_call_id='5130b1c9-fba5-41fe-850c-6e07692f28fc'),
  AIMessage(content='', additional_kwargs={'function_call': {'name': 'converter', 'arguments': '{"base_currency": 100, "conversion_rate": 88.6496}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--3efc2acb-5e76-4498-9704-3d97c4300703-0', tool_calls=[{'name': 'converter', 'args': {'base_currency': 100, 'conversion_rate': 88.6496}, 'id': '659bbe42-f82f-490b-b45d-f25e9b40fd25', 'type': 'tool_call'}], usage_metadata={'input_tokens': 145, 'output_tokens': 9, 'total_tokens': 154, 'input_token_details': {'cache_read': 0}}),
  ToolMessage(content='8864.960000000001', name='converter', id='1ef7bda3-7e03-4f1c-be95-ce2e223a9801', tool_call_id='659bbe42-f82f-490b-b45d-f25e9b40fd25'),
  AIMessage(content='100 USD is 8864.96 INR.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--f3c0322b-f4a6-43f0-bce9-a062de019a59-0', usage_metadata={'input_tokens': 157, 'output_tokens': 16, 'total_tokens': 173, 'input_token_details': {'cache_read': 0}})]}
"""

In [24]:
query = "convert 1000 INR into dirham"

response = agent.invoke(
	input={"messages": [HumanMessage(content=query)]}
	)

print(response['messages'][-1].content)

1000 INR is 41.43 dirham.
